In [1]:
import copy
import numpy as np
import os

import pickle

import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import time

from torch.autograd import Variable

# from extractor import get_features
# from extractor
os.listdir()

data_dir = 'data'
train_np_dir = 'train_np_med'
test_np_dir = 'test_np_med'
trained_weights_dir = 'trained_weights'

In [2]:
# Load existing clasess
pkl_file = open('classes_dict.pickle', 'rb')

classes_dict= pickle.load(pkl_file)

pkl_file.close()

print(classes_dict['PlayingGuitar'], len(classes_dict))

62 101


In [3]:
hidden_size = 512
input_size = 512
num_layers = 1
dropout = 0.75
batch_size = 128
num_epochs = 200
learning_rate = 0.0004
weight_decay = 0.00005
lr_decay_epoch = 5
num_classes = len(classes_dict)
sequence_length = 50
# label_str_to_int = {'ApplyEyeMakeup': 0, 'Archery': 1, 'ApplyLipstick': 2}

#Very best params (72.85% on np_big):
# hidden_size = 512
# input_size = 512
# num_layers = 1
# dropout = 0.8
# batch_size = 128
# num_epochs = 200
# learning_rate = 0.0004
# weight_decay = 0.00005

### Define models

In [4]:
# RNN Model (Many-to-One)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial states 
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).cuda()) 
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).cuda())
        
        out, _ = self.lstm(x, (h0, c0))
        
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])  
        return out

rnn = RNN(input_size=512, hidden_size=hidden_size,
          num_layers=num_layers, num_classes=num_classes, dropout=dropout).cuda()
rnn


RNN (
  (lstm): LSTM(512, 512, batch_first=True, dropout=0.75)
  (fc): Linear (512 -> 101)
)

In [5]:
from bnlstm import LSTM, LSTMCell, BNLSTMCell
# RNN Model (Many-to-One)
class ConvLSTMBatchNorm(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout):
        super(ConvLSTMBatchNorm, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = LSTM(cell_class=BNLSTMCell, input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout, max_length=input_size).cuda()
        self.fc = nn.Linear(hidden_size, num_classes).cuda()
    
    def forward(self, x):
        # Set initial states
        h0 = Variable(torch.zeros(x.size(0), self.hidden_size).cuda())
        c0 = Variable(torch.zeros(x.size(0), self.hidden_size).cuda())
        
        _, (out, _) = self.lstm(input_=x, hx=(h0, c0))
                                  
        # Decode hidden state of last time step
        out = self.fc(out[0])
        return out

rnn_batch = ConvLSTMBatchNorm(input_size=512, hidden_size=hidden_size,
          num_layers=num_layers, num_classes=num_classes, dropout=dropout).cuda()
rnn_batch

ConvLSTMBatchNorm (
  (lstm): LSTM (
    (cell_0): BNLSTMCell (
      (bn_ih): SeparatedBatchNorm1d(2048, eps=1e-05, momentum=0.1, max_length=512, affine=True)
      (bn_hh): SeparatedBatchNorm1d(2048, eps=1e-05, momentum=0.1, max_length=512, affine=True)
      (bn_c): SeparatedBatchNorm1d(512, eps=1e-05, momentum=0.1, max_length=512, affine=True)
    )
    (dropout_layer): Dropout (p = 0.75)
  )
  (fc): Linear (512 -> 101)
)

In [6]:
use_gpu = torch.cuda.is_available()
print ("GPU is available: ", use_gpu)

GPU is available:  True


In [7]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn_batch.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [8]:
def exp_lr_scheduler(optimizer, epoch, init_lr=learning_rate, lr_decay_epoch=lr_decay_epoch):
    """Decay learning rate by a factor of 0.1 every lr_decay_epoch epochs."""
    lr = init_lr * (0.5**(epoch // lr_decay_epoch))
#     lr = init_lr #* 1 / (1 + )
    if epoch % lr_decay_epoch == 0:
        print('LR is set to {}'.format(lr))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return optimizer


In [9]:
def copute_dataset_size(path=''):
    data_classes = [i for i in os.listdir(path) if not i.startswith('.')]
    num_entries = 0

    for folder in data_classes:
        current_dir = path + '/' + folder + '/'
        num_entries += len(os.listdir(current_dir))
        
    return num_entries

train_size = copute_dataset_size(path=data_dir + '/' + train_np_dir)
test_size = copute_dataset_size(path=data_dir + '/' + test_np_dir)

def check_size(address, sequence_size=50):
    x = np.load(address)
    print (x.shape)
    if x.shape[0] < sequence_size:
        return np.concatenate((x, np.zeros((sequence_size - x.shape[0], x.shape[1]))), axis=0)
    return x

# Data Loader (Input Pipeline)
def get_loader(path='', batch_size=batch_size):
    """
    Function reads .npy files from path.
    Returns:
        dataloader, data classes (list), size of input object [n_sequence, n_features], lenght_of_dataset
    """
    inputs = []
    targets = []
    data_classes = [i for i in os.listdir(path) if not i.startswith('.')]

    for folder in data_classes:
        current_dir = path + '/' + folder + '/'
        files = os.listdir(current_dir)
        #test_f = np.load(current_dir + files[0])[:,:30,:]
        
#         print(test_f.shape)
        temp = [torch.Tensor(np.load(current_dir +  f).reshape((sequence_length, input_size))) for f in files] 
                         # Transform to torch tensors
        
        targets += ([torch.LongTensor([classes_dict[folder]])] * len(temp))
        inputs += temp

    tensor_x = torch.stack(inputs)
    tensor_y = torch.stack(targets)
    my_dataset = torch.utils.data.TensorDataset(tensor_x, tensor_y)  # Create your datset
    my_dataloader = torch.utils.data.DataLoader(my_dataset, batch_size=batch_size, shuffle=True)  # Create your dataloader

    return (my_dataloader, data_classes)

dset_loaders = {x: get_loader(data_dir + '/' + x)[0] for x in [train_np_dir, test_np_dir]}

In [10]:
# os.listdir(data_dir + '/' + test_np_dir)

In [11]:
dset_sizes = {train_np_dir: train_size, test_np_dir: test_size}
dset_sizes

{'test_np_med': 3783, 'train_np_med': 9522}

In [12]:
def train_model(model, criterion, optimizer, lr_scheduler, path, num_epochs=200, model_name='lstm'):
    since = time.time()

    best_model = model
    best_acc = 0.0
    training_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in [train_np_dir, test_np_dir]:
            if phase == train_np_dir:
                optimizer = lr_scheduler(optimizer, epoch)
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            counter = 0
            # Iterate over data.
            for data in dset_loaders[phase]:
                # get the inputs

                inputs, labels = data

                # wrap them in Variable
                if use_gpu:
#                     print(inputs)
                    inputs = Variable(inputs.view(-1, sequence_length, input_size).cuda())
                    labels = Variable(labels.view(-1).cuda())                        
                else:
                    inputs, labels = Variable(inputs), Variable(labels)
                    
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
#                 print (inputs.size())
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)
#                 print (labels, outputs)

                # backward + optimize only if in training phase
                if phase == train_np_dir:
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data[0]
                running_corrects += torch.sum(preds == labels.data)

#             print(preds, labels.data, '\n========')
            epoch_loss = running_loss / dset_sizes[phase]
            epoch_acc = running_corrects / dset_sizes[phase]

            print('running corrects: ', running_corrects)
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == train_np_dir:
                training_acc = epoch_acc
            
            # deep copy the model
            if phase == test_np_dir and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model = copy.deepcopy(model)

        # saving weights
        torch.save(model, data_dir + '/' + trained_weights_dir + "/" + model_name + '_' + str(epoch) + ".pt")
        if (training_acc == 1):
            print ('accuracy 1 reached, stopping...', training_acc)
            break

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    return best_model


In [13]:
model_lstm = train_model(rnn_batch, criterion, optimizer, exp_lr_scheduler, '', num_epochs=num_epochs, model_name='lstm')

Epoch 0/199
----------
LR is set to 0.0004
running corrects:  3523
train_np_med Loss: 0.0341 Acc: 0.3700
running corrects:  1724
test_np_med Loss: 0.0315 Acc: 0.4557


/Users/philip/.virtualenvs/cv3/lib/python3.6/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type ConvLSTMBatchNorm. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch 1/199
----------
running corrects:  5750
train_np_med Loss: 0.0275 Acc: 0.6039
running corrects:  2024
test_np_med Loss: 0.0234 Acc: 0.5350
Epoch 2/199
----------
running corrects:  6909
train_np_med Loss: 0.0182 Acc: 0.7256
running corrects:  2260
test_np_med Loss: 0.0165 Acc: 0.5974
Epoch 3/199
----------
running corrects:  8023
train_np_med Loss: 0.0102 Acc: 0.8426
running corrects:  2466
test_np_med Loss: 0.0131 Acc: 0.6519
Epoch 4/199
----------
running corrects:  8833
train_np_med Loss: 0.0053 Acc: 0.9276
running corrects:  2560
test_np_med Loss: 0.0112 Acc: 0.6767
Epoch 5/199
----------
LR is set to 0.0002
running corrects:  9271
train_np_med Loss: 0.0027 Acc: 0.9736
running corrects:  2663
test_np_med Loss: 0.0100 Acc: 0.7039
Epoch 6/199
----------
running corrects:  9401
train_np_med Loss: 0.0017 Acc: 0.9873
running corrects:  2708
test_np_med Loss: 0.0099 Acc: 0.7158
Epoch 7/199
----------
running corrects:  9451
train_np_med Loss: 0.0012 Acc: 0.9925
running corrects:  

In [ ]:
for data in get_loader(data_dir + '/' + test_np_dir, batch_size=3000)[0]:
    # get the inputs

    inputs, labels = data

    # wrap them in Variable
    if use_gpu:
        x_test = Variable(inputs.view(-1, sequence_length, input_size).cuda())
        y_test = labels.view(-1).cpu().numpy()
        y_pred = model_lstm(x_test)
        _, y_pred = torch.max(y_pred.data, 1)
        y_pred = y_pred.cpu().view(-1).numpy()
    break
# y_pred

In [ ]:

import itertools
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize=(20, 20))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    print([round(cm[i][i], 2) for i in range(0, num_classes)])

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
# plt.figure()
class_int_to_name = {classes_dict[key]:key for key in classes_dict}
names_of_classes = [class_int_to_name[i] for i in range(0, num_classes)]
print(len(names_of_classes), len(y_test))
# plot_confusion_matrix(cnf_matrix, classes=names_of_classes,
#                       title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
# plt.figure()
plot_confusion_matrix(cnf_matrix, classes=names_of_classes, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

In [ ]:
# class_int_to_name = {classes_dict[key]:key for key in classes_dict}
# class_int_to_name

In [ ]:
# # Test set is not implemented yet

# # Test the Model
# correct = 0
# total = 0
# for images, labels in test_loader:
#     images = Variable(images.view(-1, sequence_length, input_size))
#     outputs = rnn(images)
#     _, predicted = torch.max(outputs.data, 1)
#     total += labels.size(0)
#     correct += (predicted == labels).sum()

# print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total)) 

# # Save the Model
# # torch.save(rnn.state_dict(), 'rnn.pkl')
# # torch.save(rnn, 'rnn.pt')

# Generate dummy data set

In [ ]:
# import numpy as np

In [ ]:
# n_feat = 100
# n_seq = 50 # fixed for now
# n_video = 10

In [ ]:
# for i in range(4,5):
#     tmp_data = np.random.random((n_seq+1, n_feat))
#     np.save('data/features/Archery/v' + str(i) + '.npy', tmp_data)